In [1]:
import os
os.environ['HTTP_PROXY'] = "http://127.0.0.1:10809"
os.environ['HTTPS_PROXY'] = "http://127.0.0.1:10809"

In [2]:
import geemap
import ee
Map=geemap.Map()
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [ ]:
# # 大庆市
# roi = ee.Geometry.Rectangle([123.9355, 45.8362,125.3055, 46.8762])
# # 天津市
# roi = ee.Geometry.Rectangle([115.9244, 38.6272,117.4944,39.6372])
# # 武汉市
# roi = ee.Geometry.Rectangle([113.7393, 29.8642,115.0993, 30.9242])
# # 珠三角
# roi = ee.Geometry.Rectangle([112.7614, 22.2347,114.0514, 23.2547])
# 青藏高原
roi = ee.Geometry.Rectangle([89.8826, 34.6579,91.3626, 35.8279])
Map.addLayer(roi, {}, "roi")
Map.centerObject(roi,7)

In [ ]:
# l8 = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR').filterBounds(roi).filterDate('2020-08-01','2020-10-31').filter(ee.Filter.lt('CLOUD_COVER',20))
# visParams = {'bands': ['B5', 'B6', 'B4'],'min': 0,'max': 3000,'gamma': 1.4}
# # Map.addLayer(l8, visParams,'image mosaic')
# print(l8.aggregate_array('system:index').getInfo())

landsat5 = ee.ImageCollection('LANDSAT/LT05/C01/T1_SR').select(['B1', 'B2', 'B3', 'B4', 'B5', 'B7','pixel_qa'],['B2', 'B3', 'B4', 'B5', 'B6', 'B7','pixel_qa'])
l5 = landsat5.filterBounds(roi).filterDate('2000-08-01','2000-10-31').filter(ee.Filter.lt('CLOUD_COVER',10))
print(l5.aggregate_array('system:index').getInfo())

a = ee.Image('LANDSAT/LC08/C01/T1_SR/LC08_122033_20200828')
visParams = {'bands': ['B5', 'B6', 'B4'],'min': 0,'max': 3000,'gamma': 1.4}
Map.addLayer(a, visParams,'a')

## 循环，生成总的样本集

In [ ]:
## 异常值去除
bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7','NDVI','NDWI','mNDWI','CWI','AWEI','EWI','EVI']
#### cluster与waterclass的对比
def Compare(feature):
    cluster = ee.Number(feature.get('cluster'))
    waterclass = ee.Number(feature.get('waterclass'))
    ft = ee.Algorithms.If(cluster.eq(waterclass),feature.set({'eq':1}),feature.set({'eq':0}))
    return ft
#### k-mean聚类
def k_mean(sampleSET):
    clusterer = ee.Clusterer.wekaKMeans(2).train(sampleSET,bands)
    result = sampleSET.cluster(clusterer)
    right = result.map(Compare).filter(ee.Filter.eq('eq',0))
    error = result.map(Compare).filter(ee.Filter.eq('eq',1))
    filtered_sample = ee.FeatureCollection(ee.Algorithms.If(right.size().gt(error.size()),right,error))
    return filtered_sample

In [ ]:
# region = 'tibet'
# sample_region = 'TIBET'
# for year in range(2000,2021,1):
#     print(year)
#     sw = ee.FeatureCollection('users/311605001111/SAMPLE_' + sample_region + '/' + region + '_kmean100_season' + str(year)+'_v2').randomColumn('random',1,'uniform').sort('random').limit(500)
#     print('抽样的季节性水体： ',sw.size().getInfo())

In [ ]:
region = 'tibet'
sample_region = 'TIBET'
totalSamples = ee.FeatureCollection([])
for year in range(1999,2021,5):
    print(year)
    ## 永久性水体样本和非水体样本，抽样
    # pw_nw = ee.FeatureCollection('users/311605001111/' + region + '_kmean100_noseason' + str(year))
    pw_nw = ee.FeatureCollection('users/311605001111/SAMPLE_' + sample_region + '/' + region + '_kmean100_noseason' + str(year))
    permanent_water = pw_nw.filter(ee.Filter.eq('waterclass',1)).randomColumn('random',1,'uniform').sort('random').limit(500)
    no_water = pw_nw.filter(ee.Filter.eq('waterclass',0)).randomColumn('random',1,'uniform').sort('random').limit(3000)
    print('抽样的永久性水体： ',permanent_water.size().getInfo())
    print('抽样的非水体： ',no_water.size().getInfo())
    ## 季节性样本，抽样
    sw = ee.FeatureCollection('users/311605001111/SAMPLE_' + sample_region + '/' + region + '_kmean100_season' + str(year)+'_v2').randomColumn('random',1,'uniform').sort('random').limit(500)
    # sw = ee.FeatureCollection('users/311605001111/SAMPLE_' + sample_region + '/' + region + '_kmean100_season' + str(year)).randomColumn('random',1,'uniform').sort('random').limit(500)
    print('抽样的季节性水体： ',sw.size().getInfo())
    ## 季节性样本，分类
    bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7','NDVI','NDWI','mNDWI','CWI','AWEI','EWI','EVI']
    cluster_samples = ee.FeatureCollection([permanent_water,no_water]).flatten()
    clusterer = ee.Clusterer.wekaKMeans(2).train(cluster_samples,bands)
    q = ee.FeatureCollection(sw.cluster(clusterer))
    q1 = q.filter(ee.Filter.eq('cluster',1))
    q0 = q.filter(ee.Filter.eq('cluster',0))
    b1 = ee.FeatureCollection(q1).filter(ee.Filter.lt('AWEI',0)).size()
    b2 = ee.FeatureCollection(q1).filter(ee.Filter.gt('AWEI',0)).size()
    c1 = ee.FeatureCollection(q0).filter(ee.Filter.lt('AWEI',0)).size()
    c2 = ee.FeatureCollection(q0).filter(ee.Filter.gt('AWEI',0)).size()
    qq1 = ee.Algorithms.If(ee.Number(b1).lt(ee.Number(b2)),q1.map(lambda i:i.set({'waterclass':1})),q1.map(lambda i:i.set({'waterclass':0})))
    qq0 = ee.Algorithms.If(ee.Number(c1).lt(ee.Number(c2)),q0.map(lambda i:i.set({'waterclass':1})),q0.map(lambda i:i.set({'waterclass':0})))
    seasonWater = ee.FeatureCollection([qq1,qq0]).flatten()
    ## 合并
    total_Samples = ee.FeatureCollection([permanent_water,no_water,seasonWater]).flatten()
    print("筛选前，剩余的样本数： ",total_Samples.size().getInfo())
    ## 异常值去除
    filtered_samples = k_mean(total_Samples)
    print("筛选后，剩余的样本数： ",filtered_samples.size().getInfo())
    print('no-water')
    print(filtered_samples.filter(ee.Filter.eq('waterclass',0)).filter(ee.Filter.lt('AWEI',0)).size().getInfo())
    print(filtered_samples.filter(ee.Filter.eq('waterclass',0)).filter(ee.Filter.gt('AWEI',0)).size().getInfo())
    print('water')
    print(filtered_samples.filter(ee.Filter.eq('waterclass',1)).filter(ee.Filter.lt('AWEI',0)).size().getInfo())
    print(filtered_samples.filter(ee.Filter.eq('waterclass',1)).filter(ee.Filter.gt('AWEI',0)).size().getInfo())
    totalSamples = totalSamples.merge(filtered_samples)
    print("总样本： ",totalSamples.size().getInfo())

In [ ]:
dataset_id = region + '_kmean100_9920_v2'
assetID = 'users/311605001111/SAMPLE_' + sample_region + '/' + dataset_id
task = ee.batch.Export.table.toAsset(**{
    'collection': totalSamples,
    'description': dataset_id,
    'assetId': assetID
})
task.start()

## 分年份生成样本集

In [ ]:
## 异常值去除
bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7','NDVI','NDWI','mNDWI','CWI','AWEI','EWI','EVI']
#### cluster与waterclass的对比
def Compare(feature):
    cluster = ee.Number(feature.get('cluster'))
    waterclass = ee.Number(feature.get('waterclass'))
    ft = ee.Algorithms.If(cluster.eq(waterclass),feature.set({'eq':1}),feature.set({'eq':0}))
    return ft
#### k-mean聚类
def k_mean(sampleSET):
    clusterer = ee.Clusterer.wekaKMeans(2).train(sampleSET,bands)
    result = sampleSET.cluster(clusterer)
    right = result.map(Compare).filter(ee.Filter.eq('eq',0))
    error = result.map(Compare).filter(ee.Filter.eq('eq',1))
    filtered_sample = ee.FeatureCollection(ee.Algorithms.If(right.size().gt(error.size()),right,error))
    return filtered_sample

In [ ]:
region = 'tibet'
sample_region = 'TIBET'
for year in range(1999,2000,5):
    print(year)
    ## 永久性水体样本和非水体样本，抽样
    # pw_nw = ee.FeatureCollection('users/311605001111/' + region + '_kmean100_noseason' + str(year))
    pw_nw = ee.FeatureCollection('users/311605001111/SAMPLE_' + sample_region + '/' + region + '_kmean100_noseason' + str(year))
    permanent_water = pw_nw.filter(ee.Filter.eq('waterclass',1)).randomColumn('random',1,'uniform').sort('random').limit(500)
    no_water = pw_nw.filter(ee.Filter.eq('waterclass',0)).randomColumn('random',1,'uniform').sort('random').limit(3000)
    print('抽样的永久性水体： ',permanent_water.size().getInfo())
    print('抽样的非水体： ',no_water.size().getInfo())
    ## 季节性样本，抽样
    sw = ee.FeatureCollection('users/311605001111/SAMPLE_' + sample_region + '/' + region + '_kmean100_season' + str(year)+'_v2').randomColumn('random',1,'uniform').sort('random').limit(500)
    # sw = ee.FeatureCollection('users/311605001111/SAMPLE_' + sample_region + '/' + region + '_kmean100_season' + str(year)).randomColumn('random',1,'uniform').sort('random').limit(500)
    print('抽样的季节性水体： ',sw.size().getInfo())
    ## 季节性样本，分类
    bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7','NDVI','NDWI','mNDWI','CWI','AWEI','EWI','EVI']
    cluster_samples = ee.FeatureCollection([permanent_water,no_water]).flatten()
    clusterer = ee.Clusterer.wekaKMeans(2).train(cluster_samples,bands)
    q = ee.FeatureCollection(sw.cluster(clusterer))
    q1 = q.filter(ee.Filter.eq('cluster',1))
    q0 = q.filter(ee.Filter.eq('cluster',0))
    b1 = ee.FeatureCollection(q1).filter(ee.Filter.lt('AWEI',0)).size()
    b2 = ee.FeatureCollection(q1).filter(ee.Filter.gt('AWEI',0)).size()
    c1 = ee.FeatureCollection(q0).filter(ee.Filter.lt('AWEI',0)).size()
    c2 = ee.FeatureCollection(q0).filter(ee.Filter.gt('AWEI',0)).size()
    qq1 = ee.Algorithms.If(ee.Number(b1).lt(ee.Number(b2)),q1.map(lambda i:i.set({'waterclass':1})),q1.map(lambda i:i.set({'waterclass':0})))
    qq0 = ee.Algorithms.If(ee.Number(c1).lt(ee.Number(c2)),q0.map(lambda i:i.set({'waterclass':1})),q0.map(lambda i:i.set({'waterclass':0})))
    seasonWater = ee.FeatureCollection([qq1,qq0]).flatten()
    ## 合并
    total_Samples = ee.FeatureCollection([permanent_water,no_water,seasonWater]).flatten()
    print("筛选前，剩余的样本数： ",total_Samples.size().getInfo())
    ## 异常值去除
    filtered_samples = k_mean(total_Samples)
    print("筛选后，剩余的样本数： ",filtered_samples.size().getInfo())
    dataset_id = region + str(year)
    assetID = 'users/311605001111/SAMPLE_' + sample_region + '/' + dataset_id
    task = ee.batch.Export.table.toAsset(**{
        'collection': filtered_samples,
        'description': dataset_id,
        'assetId': assetID
    })
    task.start()

### 验证

In [ ]:
region = 'tibet'
sample_region = 'TIBET'
totalSamples = ee.FeatureCollection([])
for year in range(1999,2021,5):
    print(year)
    pw_nw = ee.FeatureCollection('users/311605001111/SAMPLE_' + sample_region + '/' + region + str(year))
    print('xxxxxx： ',pw_nw.size().getInfo())
    totalSamples = totalSamples.merge(pw_nw)
    print("总样本： ",totalSamples.size().getInfo())

## 单个

In [ ]:
region = 'tibet'
sample_region = 'TIBET'
year = 1999

## 永久性水体样本和非水体样本，抽样
pw_nw = ee.FeatureCollection('users/311605001111/SAMPLE_' + sample_region + '/' + region + '_kmean100_noseason' + str(year))
permanent_water = pw_nw.filter(ee.Filter.eq('waterclass',1)).randomColumn('random',1,'uniform').sort('random').limit(500)
no_water = pw_nw.filter(ee.Filter.eq('waterclass',0)).randomColumn('random',1,'uniform').sort('random').limit(3000)
print('抽样的永久性水体： ',permanent_water.size().getInfo())
print('抽样的非水体： ',no_water.size().getInfo())

## 季节性样本，抽样
sw = ee.FeatureCollection('users/311605001111/SAMPLE_' + sample_region + '/' + region + '_kmean100_season' + str(year)+'_v3').randomColumn('random',1,'uniform').sort('random').limit(500)
# sw = ee.FeatureCollection('users/311605001111/SAMPLE_' + sample_region + '/' + region + '_kmean100_season' + str(year)).randomColumn('random',1,'uniform').sort('random').limit(500)
print('抽样的季节性水体： ',sw.size().getInfo())

## 季节性样本，分类
bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7','NDVI','NDWI','mNDWI','CWI','AWEI','EWI','EVI']
cluster_samples = ee.FeatureCollection([permanent_water,no_water]).flatten()
clusterer = ee.Clusterer.wekaKMeans(2).train(cluster_samples,bands)
q = ee.FeatureCollection(sw.cluster(clusterer))
q1 = q.filter(ee.Filter.eq('cluster',1))
q0 = q.filter(ee.Filter.eq('cluster',0))
b1 = ee.FeatureCollection(q1).filter(ee.Filter.lt('AWEI',0)).size()
b2 = ee.FeatureCollection(q1).filter(ee.Filter.gt('AWEI',0)).size()
c1 = ee.FeatureCollection(q0).filter(ee.Filter.lt('AWEI',0)).size()
c2 = ee.FeatureCollection(q0).filter(ee.Filter.gt('AWEI',0)).size()

qq1 = ee.Algorithms.If(ee.Number(b1).lt(ee.Number(b2)),q1.map(lambda i:i.set({'waterclass':1})),q1.map(lambda i:i.set({'waterclass':0})))
qq0 = ee.Algorithms.If(ee.Number(c1).lt(ee.Number(c2)),q0.map(lambda i:i.set({'waterclass':1})),q0.map(lambda i:i.set({'waterclass':0})))
seasonWater = ee.FeatureCollection([qq1,qq0]).flatten()

## 合并
total_Samples = ee.FeatureCollection([permanent_water,no_water,seasonWater]).flatten()
print("筛选前，总的样本数： ",total_Samples.size().getInfo())

## 异常值去除
bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7','NDVI','NDWI','mNDWI','CWI','AWEI','EWI','EVI']
#### cluster与waterclass的对比
def Compare(feature):
    cluster = ee.Number(feature.get('cluster'))
    waterclass = ee.Number(feature.get('waterclass'))
    ft = ee.Algorithms.If(cluster.eq(waterclass),feature.set({'eq':1}),feature.set({'eq':0}))
    return ft
#### k-mean聚类
def k_mean(sampleSET):
    clusterer = ee.Clusterer.wekaKMeans(2).train(sampleSET,bands)
    result = sampleSET.cluster(clusterer)
    right = result.map(Compare).filter(ee.Filter.eq('eq',0))
    error = result.map(Compare).filter(ee.Filter.eq('eq',1))
    filtered_sample = ee.FeatureCollection(ee.Algorithms.If(right.size().gt(error.size()),right,error))
    return filtered_sample

filtered_samples = k_mean(total_Samples)
print("筛选后，剩余的样本数： ",filtered_samples.size().getInfo())

print('no-water')
print(filtered_samples.filter(ee.Filter.eq('waterclass',0)).filter(ee.Filter.lt('AWEI',0)).size().getInfo())
print(filtered_samples.filter(ee.Filter.eq('waterclass',0)).filter(ee.Filter.gt('AWEI',0)).size().getInfo())
print('water')
print(filtered_samples.filter(ee.Filter.eq('waterclass',1)).filter(ee.Filter.lt('AWEI',0)).size().getInfo())
print(filtered_samples.filter(ee.Filter.eq('waterclass',1)).filter(ee.Filter.gt('AWEI',0)).size().getInfo())

print('no-water')
print(filtered_samples.filter(ee.Filter.eq('waterclass',0)).filter(ee.Filter.lt('mNDWI',0)).size().getInfo())
print(filtered_samples.filter(ee.Filter.eq('waterclass',0)).filter(ee.Filter.gt('mNDWI',0)).size().getInfo())
print('water')
print(filtered_samples.filter(ee.Filter.eq('waterclass',1)).filter(ee.Filter.lt('mNDWI',0)).size().getInfo())
print(filtered_samples.filter(ee.Filter.eq('waterclass',1)).filter(ee.Filter.gt('mNDWI',0)).size().getInfo())

In [ ]:
region = 'daqing'
sample_region = 'DAQING'
year = 2019

## 永久性水体样本和非水体样本，抽样
pw_nw = ee.FeatureCollection('users/311605001111/SAMPLE_' + sample_region + '/' + region + '_kmean100_noseason' + str(year))
permanent_water = pw_nw.filter(ee.Filter.eq('waterclass',1)).randomColumn('random',1,'uniform').sort('random').limit(500)
no_water = pw_nw.filter(ee.Filter.eq('waterclass',0)).randomColumn('random',1,'uniform').sort('random').limit(3000)
print('抽样的永久性水体： ',permanent_water.size().getInfo())
print('抽样的非水体： ',no_water.size().getInfo())

## 季节性样本，抽样
# sw = ee.FeatureCollection('users/311605001111/SAMPLE_' + sample_region + '/' + region + '_kmean100_season' + str(year)+'_v2').randomColumn('random',1,'uniform').sort('random').limit(500)
sw = ee.FeatureCollection('users/311605001111/SAMPLE_' + sample_region + '/' + region + '_kmean100_season' + str(year)).randomColumn('random',1,'uniform').sort('random').limit(500)
print('抽样的季节性水体： ',sw.size().getInfo())

## 季节性样本，分类
bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7','NDVI','NDWI','mNDWI','CWI','AWEI','EWI','EVI']
cluster_samples = ee.FeatureCollection([permanent_water,no_water]).flatten()

# clusterer = ee.Clusterer.wekaKMeans(2).train(cluster_samples,bands)
clusterer = ee.Clusterer.wekaKMeans(2).train(sw,bands)

q = ee.FeatureCollection(sw.cluster(clusterer))
q1 = q.filter(ee.Filter.eq('cluster',1))
q0 = q.filter(ee.Filter.eq('cluster',0))
b1 = ee.FeatureCollection(q1).filter(ee.Filter.lt('AWEI',0)).size()
b2 = ee.FeatureCollection(q1).filter(ee.Filter.gt('AWEI',0)).size()
c1 = ee.FeatureCollection(q0).filter(ee.Filter.lt('AWEI',0)).size()
c2 = ee.FeatureCollection(q0).filter(ee.Filter.gt('AWEI',0)).size()
# b1 = ee.FeatureCollection(q1).filter(ee.Filter.lt('mNDWI',0)).size()
# b2 = ee.FeatureCollection(q1).filter(ee.Filter.gt('mNDWI',0)).size()
# c1 = ee.FeatureCollection(q0).filter(ee.Filter.lt('mNDWI',0)).size()
# c2 = ee.FeatureCollection(q0).filter(ee.Filter.gt('mNDWI',0)).size()

qq1 = ee.Algorithms.If(ee.Number(b1).lt(ee.Number(b2)),q1.map(lambda i:i.set({'waterclass':1})),q1.map(lambda i:i.set({'waterclass':0})))
qq0 = ee.Algorithms.If(ee.Number(c1).lt(ee.Number(c2)),q0.map(lambda i:i.set({'waterclass':1})),q0.map(lambda i:i.set({'waterclass':0})))
seasonWater = ee.FeatureCollection([qq1,qq0]).flatten()

## 合并
total_Samples = ee.FeatureCollection([permanent_water,no_water,seasonWater]).flatten()
print("筛选前，总的样本数： ",total_Samples.size().getInfo())

print('season')
print('no-water')
print(seasonWater.filter(ee.Filter.eq('waterclass',0)).filter(ee.Filter.lt('AWEI',0)).size().getInfo())
print(seasonWater.filter(ee.Filter.eq('waterclass',0)).filter(ee.Filter.gt('AWEI',0)).size().getInfo())
print('water')
print(seasonWater.filter(ee.Filter.eq('waterclass',1)).filter(ee.Filter.lt('AWEI',0)).size().getInfo())
print(seasonWater.filter(ee.Filter.eq('waterclass',1)).filter(ee.Filter.gt('AWEI',0)).size().getInfo())

print('no-water')
print(seasonWater.filter(ee.Filter.eq('waterclass',0)).filter(ee.Filter.lt('mNDWI',0)).size().getInfo())
print(seasonWater.filter(ee.Filter.eq('waterclass',0)).filter(ee.Filter.gt('mNDWI',0)).size().getInfo())
print('water')
print(seasonWater.filter(ee.Filter.eq('waterclass',1)).filter(ee.Filter.lt('mNDWI',0)).size().getInfo())
print(seasonWater.filter(ee.Filter.eq('waterclass',1)).filter(ee.Filter.gt('mNDWI',0)).size().getInfo())

print('permanent and land')
print('no-water')
print(cluster_samples.filter(ee.Filter.eq('waterclass',0)).filter(ee.Filter.lt('AWEI',0)).size().getInfo())
print(cluster_samples.filter(ee.Filter.eq('waterclass',0)).filter(ee.Filter.gt('AWEI',0)).size().getInfo())
print('water')
print(cluster_samples.filter(ee.Filter.eq('waterclass',1)).filter(ee.Filter.lt('AWEI',0)).size().getInfo())
print(cluster_samples.filter(ee.Filter.eq('waterclass',1)).filter(ee.Filter.gt('AWEI',0)).size().getInfo())

print('no-water')
print(cluster_samples.filter(ee.Filter.eq('waterclass',0)).filter(ee.Filter.lt('mNDWI',0)).size().getInfo())
print(cluster_samples.filter(ee.Filter.eq('waterclass',0)).filter(ee.Filter.gt('mNDWI',0)).size().getInfo())
print('water')
print(cluster_samples.filter(ee.Filter.eq('waterclass',1)).filter(ee.Filter.lt('mNDWI',0)).size().getInfo())
print(cluster_samples.filter(ee.Filter.eq('waterclass',1)).filter(ee.Filter.gt('mNDWI',0)).size().getInfo())

In [ ]:
抽样的永久性水体：  500
抽样的非水体：  3000
抽样的季节性水体：  500
筛选前，总的样本数：  4000
season
no-water
311
3
water
43
143
no-water
295
19
water
17
169
permanent and land
no-water
2990
10
water
9
491
no-water
2959
41
water
7
493

# 可用于实验的影像ID--各区域

In [ ]:
# 区域 A 
'LANDSAT/LC08/C01/T1_SR/LC08_119028_20201111' 
'LANDSAT/LC08/C01/T1_SR/LC08_120027_20200526' 
'LANDSAT/LC08/C01/T1_SR/LC08_120028_20200526'

# 区域 B
'LANDSAT/LC08/C01/T1_SR/LC08_122033_20200422'
'LANDSAT/LC08/C01/T1_SR/LC08_122033_20200828'

'LANDSAT/LC08/C01/T1_SR/LC08_123033_20200904' 
'LANDSAT/LC08/C01/T1_SR/LC08_123033_20200920' 
'LANDSAT/LC08/C01/T1_SR/LC08_123033_20201022' 
'LANDSAT/LC08/C01/T1_SR/LC08_123033_20201225'

# 区域 C
'LANDSAT/LC08/C01/T1_SR/LC08_122039_20200218' 
'LANDSAT/LC08/C01/T1_SR/LC08_122039_20200305' 
'LANDSAT/LC08/C01/T1_SR/LC08_122039_20200828'
  
'LANDSAT/LC08/C01/T1_SR/LC08_123039_20200209'
'LANDSAT/LC08/C01/T1_SR/LC08_123039_20200413'
'LANDSAT/LC08/C01/T1_SR/LC08_123039_20200429'
'LANDSAT/LC08/C01/T1_SR/LC08_123039_20200803'
'LANDSAT/LC08/C01/T1_SR/LC08_123039_20201022'
'LANDSAT/LC08/C01/T1_SR/LC08_123039_20201225'

# 区域 D
'LANDSAT/LC08/C01/T1_SR/LC08_122044_20200218'
'LANDSAT/LC08/C01/T1_SR/LC08_122044_20201116'

'LANDSAT/LC08/C01/T1_SR/LC08_122045_20201202'

# 区域 E
'LANDSAT/LC08/C01/T1_SR/LC08_138036_20200929'
'LANDSAT/LC08/C01/T1_SR/LC08_138036_20201015'
'LANDSAT/LC08/C01/T1_SR/LC08_138036_20201031' 
 
'LANDSAT/LC08/C01/T1_SR/LC08_139035_20200819'
'LANDSAT/LC08/C01/T1_SR/LC08_139035_20201006'
 
'LANDSAT/LC08/C01/T1_SR/LC08_139036_20200819'
'LANDSAT/LC08/C01/T1_SR/LC08_139036_20201006'

# 计算山地阴影

In [ ]:
# 计算山地阴影
# Define a function to compute a hillshade from terrain data for the given sun azimuth and elevation.
def hillshade(image):    
    # Convert angles to radians
    azimuth = ee.Image(ee.Number(img.get('SOLAR_AZIMUTH_ANGLE'))).rename('azimuth').clip(roi).toFloat().multiply(math.pi).divide(180)
    zenith = ee.Image(ee.Number(img.get('SOLAR_ZENITH_ANGLE'))).rename('zenith').clip(roi).toFloat().multiply(math.pi).divide(180)
    # calcluate slope and aspect
    terrain = ee.Algorithms.Terrain(ee.Image('CGIAR/SRTM90_V4'))
    slope = terrain.select('slope').rename('slope').clip(roi).toFloat().multiply(math.pi).divide(180)
    aspect = terrain.select('aspect').rename('aspect').clip(roi).toFloat().multiply(math.pi).divide(180)
    # Note that methods on images are needed to do the computation.
    # The following implements: Hillshade = cos(Azimuth - Aspect) * sin(Slope) * sin(Zenith) + cos(Zenith) * cos(Slope)
    return azimuth.subtract(aspect).cos().multiply(slope.sin()).multiply(zenith.sin()).add(zenith.cos().multiply(slope.cos()))

# 计算区域高程的最值

In [ ]:
# 计算区域高程
elevation = ee.Image('USGS/SRTMGL1_003').select('elevation').clip(roi)
minmax = elevation.reduceRegion(**{
    'reducer': ee.Reducer.minMax(),
    'geometry': roi,
    'scale': 30,
    'maxPixels': 1e14,
})
print(minmax.getInfo())

# 计算“pixel_qa”的最大值和最小值

In [ ]:
def maxmin(image):
    minmax = image.select('pixel_qa').reduceRegion(**{
        'reducer': ee.Reducer.minMax(),
        'geometry': roi,
        'scale': 30,
        'maxPixels': 1e14,
    })
    return image.set({'max':minmax.get('pixel_qa_max')}).set({'min':minmax.get('pixel_qa_min')})

l7 = ee.ImageCollection('LANDSAT/LE07/C01/T1_SR') \
           .select(['B1', 'B2', 'B3', 'B4', 'B5', 'B7','pixel_qa'],['B2', 'B3', 'B4', 'B5', 'B6', 'B7','pixel_qa']) \
           .filterBounds(roi) \
           .filterDate('2014-01-01', '2020-12-31')
           
l8 = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
       .select(['B2', 'B3', 'B4', 'B5', 'B6', 'B7','pixel_qa']) \
       .filterBounds(roi) \
       .filterDate('2014-01-01', '2020-12-31')

landsat_image = l8.merge(l7).map(maxmin)
# Map.addLayer(l8.map(maskSR).median().clip(roi), visParams,'image mosaic')
print(landsat_image.aggregate_array('max').getInfo())
print(landsat_image.size().getInfo())

# 筛选影像函数

## 单个数据集

In [ ]:
# 影像筛选函数
def filter_image(id):
    l8 = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR').select(['B2', 'B3', 'B4', 'B5', 'B6', 'B7','pixel_qa']).filterBounds(roi).filterDate('2020-01-01', '2020-12-31')
    images = l8.map(lambda i:i.setMulti({'path_row': ee.String(i.get('system:index')).slice(5,11),'month':ee.Number.parse(ee.String(i.get('system:index')).slice(-4,-2))}))
    month = images.map(lambda i:i.set({'img_id':i.get('system:index')})).filter(ee.Filter.eq('path_row',id))
    first_quarter = ee.ImageCollection(month.filter(ee.Filter.lte('month',3)).filter(ee.Filter.gte('month',1)).randomColumn('random',1,'uniform').sort('random')).first()
    second_quarter = ee.ImageCollection(month.filter(ee.Filter.lte('month',6)).filter(ee.Filter.gte('month',4)).randomColumn('random',1,'uniform').sort('random')).first()
    third_quarter = ee.ImageCollection(month.filter(ee.Filter.lte('month',9)).filter(ee.Filter.gte('month',7)).randomColumn('random',1,'uniform').sort('random')).first()
    fourth_quarter = ee.ImageCollection(month.filter(ee.Filter.lte('month',12)).filter(ee.Filter.gte('month',10)).randomColumn('random',1,'uniform').sort('random')).first()
    img_col = ee.ImageCollection([first_quarter,second_quarter,third_quarter,fourth_quarter])
    return img_col

## 计算唯一的行列号
region_images = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR').filterBounds(roi).filterDate('2020-01-01', '2020-12-31')
landsat_image = region_images.map(lambda i : i.setMulti({'path_row': ee.String(i.get('system:index')).slice(5,11),'month':ee.Number.parse(ee.String(i.get('system:index')).slice(-4,-2))}))
path_row = landsat_image.distinct('path_row').aggregate_array('path_row')
print(path_row.getInfo())

## 计算在每个行列号上选择的月份
select_image = path_row.map(filter_image)
number = path_row.size().getInfo()
print(path_row.size().getInfo())
for i in range(0,number):
    print(ee.ImageCollection(select_image .get(i)).aggregate_array('month').getInfo())


## 合并在各个行列号上选择的影像
img_col = ee.ImageCollection([])
for i in range(0,number):
    img_col = img_col.merge(ee.ImageCollection(select_image.get(i)))

print(img_col.size().getInfo())
print(img_col.aggregate_array('path_row').getInfo())
print(img_col.aggregate_array('month').getInfo())

## 综合版

### 加载数据集

In [ ]:
# 定义参数
year = '2000'
startDate = year + '-01-01'
endDate = year + '-12-31'

l5 = ee.ImageCollection('LANDSAT/LT05/C01/T1_SR') \
       .select(['B1', 'B2', 'B3', 'B4', 'B5', 'B7','pixel_qa'],['B2', 'B3', 'B4', 'B5', 'B6', 'B7','pixel_qa']) \
       .filterBounds(roi) \
       .filterDate(startDate, endDate)  
l7 = ee.ImageCollection('LANDSAT/LE07/C01/T1_SR') \
       .select(['B1', 'B2', 'B3', 'B4', 'B5', 'B7','pixel_qa'],['B2', 'B3', 'B4', 'B5', 'B6', 'B7','pixel_qa']) \
       .filterBounds(roi) \
       .filterDate(startDate, endDate)           
l8 = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
       .select(['B2', 'B3', 'B4', 'B5', 'B6', 'B7','pixel_qa']) \
       .filterBounds(roi) \
       .filterDate(startDate, endDate)
landsat_images = l8.merge(l7).merge(l5)
# 图像可视化参数
visParams = {'bands': ['B5', 'B6', 'B4'],'min': 0,'max': 3000,'gamma': 1.4}
Map.addLayer(l8.mosaic().clip(roi), visParams,'image mosaic')
print(l5.size().getInfo( ))
print(l7.size().getInfo())
print(l8.size().getInfo())
print(landsat_images.size().getInfo())

### 函数定义

In [ ]:
# 影像筛选函数
def filter_image(id):
    # l8
    images_l8 = l8.map(lambda i:i.setMulti({'path_row': ee.String(i.get('system:id')).slice(-15,-9),'month':ee.Number.parse(ee.String(i.get('system:id')).slice(-4,-2))}))
    month_l8 = images_l8.map(lambda i:i.set({'img_id':i.get('system:index')})).filter(ee.Filter.eq('path_row',id))
    first_quarter_l8 = ee.ImageCollection(month_l8.filter(ee.Filter.lte('month',3)).filter(ee.Filter.gte('month',1)).randomColumn('random',1,'uniform').sort('random')).first()
    second_quarter_l8 = ee.ImageCollection(month_l8.filter(ee.Filter.lte('month',6)).filter(ee.Filter.gte('month',4)).randomColumn('random',1,'uniform').sort('random')).first()
    third_quarter_l8 = ee.ImageCollection(month_l8.filter(ee.Filter.lte('month',9)).filter(ee.Filter.gte('month',7)).randomColumn('random',1,'uniform').sort('random')).first()
    fourth_quarter_l8 = ee.ImageCollection(month_l8.filter(ee.Filter.lte('month',12)).filter(ee.Filter.gte('month',10)).randomColumn('random',1,'uniform').sort('random')).first()
    # l7
    images_l7 = l7.map(lambda i:i.setMulti({'path_row': ee.String(i.get('system:id')).slice(-15,-9),'month':ee.Number.parse(ee.String(i.get('system:id')).slice(-4,-2))}))
    month_l7 = images_l7.map(lambda i:i.set({'img_id':i.get('system:index')})).filter(ee.Filter.eq('path_row',id))
    first_quarter_l7 = ee.ImageCollection(month_l7.filter(ee.Filter.lte('month',3)).filter(ee.Filter.gte('month',1)).randomColumn('random',1,'uniform').sort('random')).first()
    second_quarter_l7 = ee.ImageCollection(month_l7.filter(ee.Filter.lte('month',6)).filter(ee.Filter.gte('month',4)).randomColumn('random',1,'uniform').sort('random')).first()
    third_quarter_l7 = ee.ImageCollection(month_l7.filter(ee.Filter.lte('month',9)).filter(ee.Filter.gte('month',7)).randomColumn('random',1,'uniform').sort('random')).first()
    fourth_quarter_l7 = ee.ImageCollection(month_l7.filter(ee.Filter.lte('month',12)).filter(ee.Filter.gte('month',10)).randomColumn('random',1,'uniform').sort('random')).first()
    # l5
    images_l5 = l5.map(lambda i:i.setMulti({'path_row': ee.String(i.get('system:id')).slice(-15,-9),'month':ee.Number.parse(ee.String(i.get('system:id')).slice(-4,-2))}))
    month_l5 = images_l5.map(lambda i:i.set({'img_id':i.get('system:index')})).filter(ee.Filter.eq('path_row',id))
    first_quarter_l5 = ee.ImageCollection(month_l5.filter(ee.Filter.lte('month',3)).filter(ee.Filter.gte('month',1)).randomColumn('random',1,'uniform').sort('random')).first()
    second_quarter_l5 = ee.ImageCollection(month_l5.filter(ee.Filter.lte('month',6)).filter(ee.Filter.gte('month',4)).randomColumn('random',1,'uniform').sort('random')).first()
    third_quarter_l5 = ee.ImageCollection(month_l5.filter(ee.Filter.lte('month',9)).filter(ee.Filter.gte('month',7)).randomColumn('random',1,'uniform').sort('random')).first()
    fourth_quarter_l5 = ee.ImageCollection(month_l5.filter(ee.Filter.lte('month',12)).filter(ee.Filter.gte('month',10)).randomColumn('random',1,'uniform').sort('random')).first()
    img_col = ee.ImageCollection([first_quarter_l8,second_quarter_l8,third_quarter_l8,fourth_quarter_l8,first_quarter_l7,second_quarter_l7,third_quarter_l7,fourth_quarter_l7,first_quarter_l5,second_quarter_l5,third_quarter_l5,fourth_quarter_l5])
    return img_col

## 计算唯一的行列号
l5 = ee.ImageCollection('LANDSAT/LT05/C01/T1_SR').filterBounds(roi).filterDate(startDate, endDate)  
l7 = ee.ImageCollection('LANDSAT/LE07/C01/T1_SR').filterBounds(roi).filterDate(startDate, endDate)           
l8 = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR').filterBounds(roi).filterDate(startDate, endDate)
region_images = l8.merge(l7).merge(l5)
landsat_image = region_images.map(lambda i : i.setMulti({'path_row': ee.String(i.get('system:id')).slice(-15,-9),'month':ee.Number.parse(ee.String(i.get('system:id')).slice(-4,-2))}))
path_row = landsat_image.distinct('path_row').aggregate_array('path_row')
print(path_row.getInfo())

## 计算在每个行列号上选择的月份和卫星类型
select_image = path_row.map(filter_image)
number = path_row.size().getInfo()
print(path_row.size().getInfo())
for i in range(0,number):
    print(ee.ImageCollection(select_image .get(i)).aggregate_array('month').getInfo())
    print(ee.ImageCollection(select_image .get(i)).aggregate_array('SATELLITE').getInfo())

## 合并在各个行列号上选择的影像
img_col = ee.ImageCollection([])
for i in range(0,number):
    img_col = img_col.merge(ee.ImageCollection(select_image.get(i)))

print(img_col.size().getInfo())
print(img_col.aggregate_array('path_row').getInfo())
print(img_col.aggregate_array('month').getInfo())

In [ ]:
print('landsat5')
for year in range(2000,2012,1):
    print(year)
    startDate = str(year) + '-01-01'
    endDate = str(year) + '-12-31'
    region_images = ee.ImageCollection('LANDSAT/LT05/C01/T1_SR').filterBounds(roi).filterDate(startDate, endDate)
    landsat_image = region_images.map(lambda i : i.setMulti({'path_row': ee.String(i.get('system:index')).slice(5,11),'month':ee.Number.parse(ee.String(i.get('system:index')).slice(-4,-2))}))
    path_row = landsat_image.distinct('path_row').aggregate_array('path_row')
    print(path_row.getInfo())
print('landsat7')
for year in range(2000,2021,1):
    print(year)
    startDate = str(year) + '-01-01'
    endDate = str(year) + '-12-31'
    region_images = ee.ImageCollection('LANDSAT/LE07/C01/T1_SR').filterBounds(roi).filterDate(startDate, endDate)
    landsat_image = region_images.map(lambda i : i.setMulti({'path_row': ee.String(i.get('system:index')).slice(5,11),'month':ee.Number.parse(ee.String(i.get('system:index')).slice(-4,-2))}))
    path_row = landsat_image.distinct('path_row').aggregate_array('path_row')
    print(path_row.getInfo())
print('landsat8')
for year in range(2013,2021,1):
    print(year)
    startDate = str(year) + '-01-01'
    endDate = str(year) + '-12-31'
    region_images = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR').filterBounds(roi).filterDate(startDate, endDate)
    landsat_image = region_images.map(lambda i : i.setMulti({'path_row': ee.String(i.get('system:index')).slice(5,11),'month':ee.Number.parse(ee.String(i.get('system:index')).slice(-4,-2))}))
    path_row = landsat_image.distinct('path_row').aggregate_array('path_row')
    print(path_row.getInfo())

# 样本集--按比例划分

In [ ]:
region = 'wuhan_1124'
year = '2020'

# 汇总
## 非水样本
nowater = ee.FeatureCollection('users/311605001111/' + region + 'noseason' + year).filter(ee.Filter.eq('waterclass',0))
nowatersample = nowater.randomColumn('random',1,'uniform').filter(ee.Filter.lt('random', 2400/nowater.size().getInfo()))
## 水样本
water = ee.FeatureCollection('users/311605001111/' + region + 'noseason' + year).filter(ee.Filter.eq('waterclass',1))
watersample = water.randomColumn('random',1,'uniform').filter(ee.Filter.lt('random', 300/water.size().getInfo()))
## 季节性水体样本
sample = ee.FeatureCollection('users/311605001111/' + region + 'season' + year)
seasonsample = sample.randomColumn('random',1,'uniform').filter(ee.Filter.lt('random', 300/sample.size().getInfo()))
## 合并
all_sample = ee.FeatureCollection([nowatersample,watersample,seasonsample]).flatten()
print(all_sample.size().getInfo())

## 计算
w = all_sample.filter(ee.Filter.eq('waterclass',1))
print(w.size().getInfo())
nw = all_sample.filter(ee.Filter.eq('waterclass',0))
print(nw.size().getInfo())
print(w.size().getInfo() / all_sample.size().getInfo())
print(nw.size().getInfo() / all_sample.size().getInfo())

## 细节

In [ ]:
q1 = ee.FeatureCollection('users/311605001111/' + region + 'noseason' + year)
q2 = ee.FeatureCollection('users/311605001111/' + region + 'season' + year)
q = ee.FeatureCollection([q1,q2]).flatten()
print(q.size().getInfo())

In [ ]:
nowater = ee.FeatureCollection('users/311605001111/' + region + 'noseason' + year).filter(ee.Filter.eq('waterclass',0))
print(nowater.size().getInfo())
nowatersample = nowater.randomColumn('random',1,'uniform').filter(ee.Filter.lt('random', 2400/nowater.size().getInfo()))
print(nowatersample.size().getInfo())

water = ee.FeatureCollection('users/311605001111/' + region + 'noseason' + year).filter(ee.Filter.eq('waterclass',1))
print(water.size().getInfo())
watersample = water.randomColumn('random',1,'uniform').filter(ee.Filter.lt('random', 300/water.size().getInfo()))
print(watersample.size().getInfo())

q2 = ee.FeatureCollection('users/311605001111/' + region + 'season' + year)
print(q2.size().getInfo())
seasonsample = q2.randomColumn('random',1,'uniform').filter(ee.Filter.lt('random', 300/q2.size().getInfo()))
print(seasonsample.size().getInfo())

all_sample = ee.FeatureCollection([nowatersample,watersample,seasonsample]).flatten()
print(all_sample.size().getInfo())
w = all_sample.filter(ee.Filter.eq('waterclass',1))
print(w.size().getInfo())
nw = all_sample.filter(ee.Filter.eq('waterclass',0))
print(nw.size().getInfo())

print(w.size().getInfo() / all_sample.size().getInfo())
print(nw.size().getInfo() / all_sample.size().getInfo())

# 计算无效像元百分比

In [ ]:
def classified_image(img):
    image = maskSR(img).select(bands).classify(trainedClassifier).eq(1).remap([0,1],[1,2]).rename('waterclass').float()
    invalidPixel = maskSR_reverse(img).select('pixel_qa').gt(0).remap([0,1],[1,0]).rename('waterclass').float()
    class_image = ee.ImageCollection([invalidPixel,image]).sum()
    invalidPixels = class_image.eq(0).multiply(ee.Image.pixelArea()).divide(1e6)
    invalidarea = invalidPixels.reduceRegion(**{'reducer': ee.Reducer.sum(),'geometry': roi,'scale': 200,'maxPixels': 1e14,'tileScale': 2}).get('waterclass')
    region = class_image.gte(0).multiply(ee.Image.pixelArea()).divide(1e6)
    regionarea = region.reduceRegion(**{'reducer': ee.Reducer.sum(),'geometry': roi,'scale': 200,'maxPixels': 1e14,'tileScale': 2}).get('waterclass')
    rate = ee.Number(invalidarea).divide(ee.Number(regionarea)).multiply(100)
    return class_image.set({'system:id':img.get('system:id')}).set({'CLOUD_COVER':img.get('CLOUD_COVER')}).set({'invalid_percentage':rate})

# 绘制直方图

In [ ]:
import collections
import matplotlib.pyplot as plt
import numpy as np
month = [3, 9, 10, 11, 11, 3, 8, 9, 12]

a = dict(collections.Counter(month))
x = list(a.keys())
y = list(a.values())

fig, ax = plt.subplots(figsize=(10, 8))
ax.bar(x=x,height=y,width=0.6,align="center",color="grey",edgecolor="red",linewidth=2.0)
ax.set_title("Distribution of selected images -- " + region + '('+ year + ')', fontsize=32)
plt.xlabel("month", fontsize=28)
plt.ylabel('number of images', fontsize=28)
plt.xticks(x,fontsize = 34)
plt.yticks([1,2,3,4],fontsize = 34)

xticks = ax.get_xticks()
for i in range(len(y)):
    xy = (xticks[i], y[i] * 1.03)
    s = str(y[i])
    ax.annotate(s=s,xy=xy,fontsize=32,color="blue",ha="center",va="baseline")
plt.show()

# 各类区域制图

In [ ]:
def maskSR(img):
    cloudShadowBitMask = (1 << 3)
    cloudsBitMask = (1 << 5) 
    qa = img.select('pixel_qa')
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0) \
                   .And(qa.bitwiseAnd(cloudsBitMask).eq(0))
    return img.updateMask(mask)
#### 设置年份
year = '2020'
JRC_id = 'JRC/GSW1_3/YearlyHistory/' + year
Maryland_id = 'users/311605001111/Maryland/Maryland_nationwide_' + year
basemap = ee.Image.constant(0).clip(roi).rename('waterclass')
# JRC
JRC = ee.Image(JRC_id).clip(roi).remap([0,1,2,3],[0,0,1,2]).rename('waterclass')
JRC_waterclass = ee.ImageCollection([JRC,basemap]).sum()
# Maryland
Maryland = ee.Image(Maryland_id).clip(roi).select('b1').rename('waterclass')
Maryland_permanent = Maryland.select('waterclass').gte(75).remap([0,1],[0,2]).rename('waterclass')
Maryland_season= ee.ImageCollection([Maryland.gte(25),Maryland.lt(75)]).sum().eq(2)
Maryland_waterclass = ee.ImageCollection([Maryland_permanent,Maryland_season,basemap]).sum()
# 三者的交集
permanentWaterExtent = ee.ImageCollection([JRC_waterclass.eq(2),Maryland_waterclass.eq(2)]).sum().eq(2)
landExtent = ee.ImageCollection([JRC_waterclass.eq(0),Maryland_waterclass.eq(0)]).sum().eq(2)
seasonWaterExtent_min = ee.ImageCollection([JRC_waterclass.eq(1),Maryland_waterclass.eq(1)]).sum().eq(2)
seasonWaterExtent_max = ee.ImageCollection([permanentWaterExtent,landExtent]).sum().remap([0,1],[1,0]).rename('waterclass')

In [ ]:
l8 = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR').filterBounds(roi).filterDate('2020-01-01','2020-12-31').map(maskSR).median().clip(roi)
visParams = {'bands': ['B5', 'B6', 'B4'],'min': 0,'max': 3000,'gamma': 1.4}
Map.addLayer(l8, visParams,'image mosaic')

permanent_points = l8.updateMask(permanentWaterExtent)
land_points = l8.updateMask(landExtent)
season_points = l8.updateMask(seasonWaterExtent_min)
Map.addLayer(permanent_points, visParams,'permanent_points')
Map.addLayer(land_points, visParams,'land_points')
Map.addLayer(season_points, visParams,'season_points')

from matplotlib import pyplot as plt
import numpy as np
import matplotlib
from geemap import cartoee
region = [113.7393, 29.8642,115.0993, 30.9242]
visParams = {'bands': ['B5', 'B4', 'B3'],'min': 0,'max': 3000,'gamma': 1.4}
refer_img = season_points.visualize(**visParams)
blend = refer_img
fig = plt.figure(figsize=(12, 8))
ax = cartoee.get_map(blend, region=region)
# label = 'permanent_points'
# ax.set_title(label = label, fontsize=28)